In [1]:
from click.core import batch
# drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!git clone https://github.com/madhueb/DIS_project1.git
import os
os.chdir('./DIS_project1')

fatal: destination path 'DIS_project1' already exists and is not an empty directory.


In [3]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 402 bytes | 402.00 KiB/s, done.
From https://github.com/madhueb/DIS_project1
   2559c82..60c5b99  main       -> origin/main
Updating 2559c82..60c5b99
Fast-forward
 src/models/dpr_index.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.8 MB/s eta 0:00:00


In [3]:
from config.config import CONFIG
from src.models.dpr_index import DPRIndexModule

print(CONFIG['index_N'])
dpr_index = DPRIndexModule(CONFIG, load_index=True)
# dpr_index = DPRIndexModule(CONFIG)

16


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/content/DIS_project1/src/models/dpr_index.py:33: FutureWarning: You are using `torch.load`

Loaded doc encodes


100%|██████████| 268022/268022 [00:00<00:00, 979313.52it/s]


Loading index


100%|██████████| 7/7 [00:34<00:00,  4.92s/it]


In [4]:
import pandas as pd

test_df = pd.read_csv(CONFIG['test_path'])

In [ ]:
import torch
from tqdm import tqdm

batch_size = CONFIG['batch_size'] * 8

top_docs_df = pd.DataFrame(columns=['id', 'docids'])

for i in tqdm(range(0, len(test_df), batch_size)):
    batch_df = test_df.iloc[i:i+batch_size]
    query = batch_df['query'].tolist()
    langs = batch_df['lang'].tolist()
    ids = batch_df['id'].tolist()
    top_docs_batch = dpr_index(query, langs)
    top_docs_df = pd.concat([top_docs_df, pd.DataFrame({'id': ids, 'docids': top_docs_batch})], ignore_index=True)


top_docs_df.to_csv(CONFIG['submit_path'], index=False)

  0%|          | 0/32 [00:00<?, ?it/s]/content/DIS_project1/src/models/dpr_index.py:126: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  chunk_tensor = torch.tensor(chunk_tensor).to(self.config['device'])
 28%|██▊       | 9/32 [01:50<04:58, 12.99s/it]

In [4]:
import pandas as pd

test_df = pd.read_csv(CONFIG['dev_path']).iloc[:100]

In [5]:
import torch
from tqdm import tqdm

batch_size = CONFIG['batch_size'] * 8

currect = 0

for i in tqdm(range(0, len(test_df), batch_size)):
    batch_df = test_df.iloc[i:i+batch_size]
    query = batch_df['query'].tolist()
    langs = batch_df['lang'].tolist()
    pos = batch_df['positive_docs'].tolist()
    top_docs_batch = dpr_index(query, langs)
    for j in range(len(top_docs_batch)):
        if pos[j] in top_docs_batch[j]:
            currect += 1
print(currect/len(test_df))


  0%|          | 0/2 [00:00<?, ?it/s]/content/DIS_project1/src/models/dpr_index.py:126: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  chunk_tensor = torch.tensor(chunk_tensor).to(self.config['device'])
100%|██████████| 2/2 [00:19<00:00,  9.68s/it]

0.0
